In [1]:
%matplotlib inline

In [2]:
# Imports
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import sys
from seaborn import color_palette
from astropy.table import Table
from astropy.io import fits

In [3]:
np.set_printoptions(threshold=sys.maxsize)

In [4]:
def landy_szalay(DD, DR, RR):
    corr_func = np.copy(DD)
    corr_func[:,2] = (np.divide(DD[:,2], RR[:,2], out=np.zeros_like(DD[:,2]), where=RR[:,2]!=0) - 2. *
                      np.divide(DR[:,2], RR[:,2], out=np.zeros_like(DR[:,2]), where=RR[:,2]!=0) + 1.)
    return corr_func


def natural(DD, DR, RR):
    corr_func = np.copy(DD)
    corr_func[:,2] = np.divide(DD[:,2], RR[:,2], out=np.zeros_like(DD[:,2]), where=RR[:,2]!=0) -  1.
    return corr_func


def L_0(x):
    return 1.


def L_2(x):
    return 0.5 * (3. * x * x - 1.)


def L_4(x):
    return (35.* x * x * x * x - 30. * x * x + 3.) / 8.


L_polynomials = [L_0, L_2, L_4]


def corr_func_multipole_calc(l, corr_func, mu_bins=100):
    L = L_polynomials[np.floor(l / 2).astype(int)]
    
    corr_func_multipole = np.zeros((np.unique(corr_func[:,0]).size, 2))
    corr_func_multipole[:,0] = np.unique(corr_func[:,0])
    for multi in corr_func_multipole:
        for corr in corr_func:
            if corr[0] == multi[0]:
    
                multi[1] += (1./ float(mu_bins)) * corr[2] * L(corr[1] + 0.5 / float(mu_bins))
    
    corr_func_multipole[:,1] = corr_func_multipole[:,1] * (2. * l + 1.)
    
    return corr_func_multipole


def corr_func_projected_calc(corr_func, dr_pi=1., r_pi_max=80.):
    corr_func_projected = np.zeros((np.unique(corr_func[:,0]).size, 2))
    corr_func_projected[:,0] = np.unique(corr_func[:,0])
    corr_func_cutoff = np.zeros((np.unique(corr_func[:,0]).size, 2))
    corr_func_cutoff[:,0] = np.unique(corr_func[:,0])
    
    for proj in corr_func_projected:
        for corr in corr_func:
            if corr[0] == proj[0] and corr[1] < r_pi_max:
                proj[1] += dr_pi * corr[2]
    
    corr_func_projected[:,1] = corr_func_projected[:,1] * 2.
    
    return corr_func_projected
    
    
def multipole_plot(DD_source, DR_source, RR_source, l, D_eff=2, R_eff=2, re_bin=False, bin_width=5,
                   estimator=landy_szalay, linestyle='-', label='', color='', linewidth=2, s_pow=2.,
                   from_file=True, normalize=True, log_scaling=False, faizan_norm=False):
    if from_file:
        print("Taking from file")
        DD = np.loadtxt(DD_source, dtype=np.float, delimiter='\t')
        DR = np.loadtxt(DR_source, dtype=np.float, delimiter='\t')
        RR = np.loadtxt(RR_source, dtype=np.float, delimiter='\t')
        
    else:
        print("Taking from source")
        DD = DD_source
        DR = DR_source
        RR = RR_source

    if normalize:
        if faizan_norm:
            DD_norm = 897523126.819702
            DR_norm = 89411043621.862991
            RR_norm = 2226723192048.839844
        
        else:
            DD_norm = (D_eff * (D_eff - 1.) / 2.)
            DR_norm = (D_eff * R_eff)
            RR_norm = (R_eff * (R_eff - 1.) / 2.)
        
        print("Normalizing")
        print("DD Normalization:", DD_norm)
        print("DR Normalization:", DR_norm)
        print("RR Normalization:", RR_norm)
        DD[:,2] = DD[:,2] / DD_norm
        DR[:,2] = DR[:,2] / DR_norm
        RR[:,2] = RR[:,2] / RR_norm

    if re_bin:
        print("Rebinning")
        DD = rebin(DD, bin_width=bin_width)
        DR = rebin(DR, bin_width=bin_width)
        RR = rebin(RR, bin_width=bin_width)
    
    corr_func = estimator(DD, DR, RR)
    corr_func_multi = corr_func_multipole_calc(l, corr_func)
     
    if log_scaling:
        log_dsep = np.log10(corr_func_multi[1,0]) - np.log10(corr_func_multi[0,0])
        seps = np.power(10., (np.log10(corr_func_multi[:,0]) + log_dsep / 2.))
        if color:
            plt.semilogx(seps, corr_func_multi[:,1]*(seps**(s_pow)),
                         linestyle=linestyle, label=label, color=color, linewidth=linewidth)
            
        else:
            plt.semilogx(seps, corr_func_multi[:,1]*(seps**(s_pow)),
                         linestyle=linestyle, label=label)
        
    else:
        dsep = corr_func_multi[1,0] - corr_func_multi[0,0]
        seps = corr_func_multi[:,0] + dsep / 2.
        if color:
            plt.plot(seps, corr_func_multi[:,1]*(seps**(s_pow)),
                         linestyle=linestyle, label=label, color=color, linewidth=linewidth)
            
        else:
            plt.plot(seps, corr_func_multi[:,1]*(seps**(s_pow)),
                         linestyle=linestyle, label=label)
            

def projected_plot(DD_source, DR_source, RR_source, D_eff=2, R_eff=2, re_bin=False, bin_width=5,
                   estimator=landy_szalay, linestyle='-', label='', color='', linewidth=2, s_pow=0.,
                   from_file=True, normalize=True, log_scaling=False, dr_pi=1., r_pi_max=80.):
    # mu separation is currently hard-coded to 0.01
    if from_file:
        print("Taking from file")
        DD = np.loadtxt(DD_source, dtype=np.float, delimiter='\t')
        DR = np.loadtxt(DR_source, dtype=np.float, delimiter='\t')
        RR = np.loadtxt(RR_source, dtype=np.float, delimiter='\t')
        
    else:
        print("Taking from source")
        DD = DD_source
        DR = DR_source
        RR = RR_source
    
    if normalize:
        DD_norm = (D_eff * (D_eff - 1.) / 2.)
        DR_norm = (D_eff * R_eff)
        RR_norm = (R_eff * (R_eff - 1.) / 2.)
        
        print("Normalizing")
        print("DD Normalization:", DD_norm)
        print("DR Normalization:", DR_norm)
        print("RR Normalization:", RR_norm)
        DD[:,2] = DD[:,2] / DD_norm
        DR[:,2] = DR[:,2] / DR_norm
        RR[:,2] = RR[:,2] / RR_norm

    if re_bin:
        print("Rebinning")
        DD = rebin(DD, bin_width=bin_width)
        DR = rebin(DR, bin_width=bin_width)
        RR = rebin(RR, bin_width=bin_width)
    
    corr_func = estimator(DD, DR, RR)
    corr_func_proj = corr_func_projected_calc(corr_func, dr_pi=dr_pi, r_pi_max=r_pi_max)
     
    if log_scaling:
        log_dr_perp = np.log10(corr_func_proj[1,0]) - np.log10(corr_func_proj[0,0])
        r_perps = np.power(10., (np.log10(corr_func_proj[:,0]) + log_dr_perp / 2.))
        if color:
            plt.semilogx(r_perps, corr_func_proj[:,1]*(r_perps**(s_pow)),
                         linestyle=linestyle, label=label, color=color, linewidth=linewidth)
            
        else:
            plt.semilogx(r_perps, corr_func_proj[:,1]*(r_perps**(s_pow)),
                         linestyle=linestyle, label=label)
        
    else:
        dr_perp = corr_func_proj[1,0] - corr_func_proj[0,0]
        r_perps = corr_func_proj[:,0] + dr_perp / 2.
        if color:
            plt.plot(r_perps, corr_func_proj[:,1]*(r_perps**(s_pow)),
                         linestyle=linestyle, label=label, color=color, linewidth=linewidth)
            
        else:
            plt.plot(r_perps, corr_func_proj[:,1]*(r_perps**(s_pow)),
                         linestyle=linestyle, label=label)

In [5]:
def smooth_covariance_matrix(cov_matrix, divisions=[0, 33, 66, 99], debug=False):
    if debug:
        np.set_printoptions(suppress=True,linewidth=np.nan,threshold=sys.maxsize)
        print("Covariance Matrix:", "\n", cov_matrix, "\n")

    cor_matrix = np.copy(cov_matrix)

    for i in range(cor_matrix.shape[0]):
        for j in range(cor_matrix.shape[1]):
            cor_matrix[i, j] = cor_matrix[i, j] / np.sqrt(cor_matrix[i, i] * cor_matrix[j, j])

    if debug:
        print("Correlation Matrix:", "\n", cor_matrix, "\n")

    for a in range(1, len(divisions)):
        for b in range(1, len(divisions)):
            for i in range(divisions[a-1], divisions[a]):
                mean = 0.
                for j in range(divisions[a] - i):
                    mean += cor_matrix[i + j, divisions[b-1] + j]
                
                mean = mean / float(divisions[a] - i)

                for j in range(divisions[a] - i):
                    cor_matrix[i + j, divisions[b-1] + j] = mean

            for i in range(divisions[b-1], divisions[b]):
                mean = 0.
                for j in range(divisions[b] - i):
                    mean += cor_matrix[divisions[a-1] + j, i + j]

                mean = mean / float(divisions[b] - i)

                for j in range(divisions[b] - i):
                    cor_matrix[divisions[a-1] + j, i + j] = mean

    if debug:
        print("Smoothed Matrix:", "\n", cor_matrix, "\n")

    for i in range(cor_matrix.shape[0]):
        for j in range(cor_matrix.shape[1]):
            cov_matrix[i, j] = cor_matrix[i, j] * np.sqrt(cov_matrix[i, i] * cov_matrix[j, j])

    if debug:
        print("Final Matrix:", "\n", cov_matrix, "\n")

    return cov_matrix


def generate_covariance_matrix(mock_dir="mocks", wp_dir="wp_corr", mps_dir="mps_corr", mean_wp_file="mean_wp_corr.dat",
                               mean_mps_file="mean_mps_corr.dat", output="../output/cov_matrix.jpg",
                               plot_cov_matrix=False, smooth_cov=True, use_mono=True, use_quad=True, use_wp=True,
                               mono_min=None, mono_max=None, quad_min=None, quad_max=None, wp_min=None, wp_max=None):

    wp_files = os.listdir("../%s/%s/" % (mock_dir, wp_dir))
    mps_files = os.listdir("../%s/%s/" % (mock_dir, mps_dir))
    wp_files.sort()
    mps_files.sort()

    M = len(wp_files)

    if use_mono:
        sample = np.loadtxt("../%s/%s/%s" % (mock_dir, mps_dir, mps_files[0]))
        
        mono_min_index = 0
        if mono_min is not None:            
            for i in range(sample.shape[0]):
                if sample[i, 0] < mono_min:
                    mono_min_index = i + 1

                else:
                    break

        mono_max_index = sample.shape[0]
        if mono_max is not None:            
            for i in range(sample.shape[0]):
                if sample[i, 0] > mono_max:
                    mono_max_index = i
                    break

        mono_bins = mono_max_index - mono_min_index

    else:
        mono_bins = 0
        mono_min_index = 0
        mono_max_index = 0

    if use_quad:
        sample = np.loadtxt("../%s/%s/%s" % (mock_dir, mps_dir, mps_files[0]))
        
        quad_min_index = 0
        if quad_min is not None:            
            for i in range(sample.shape[0]):
                if sample[i, 0] < quad_min:
                    quad_min_index = i + 1

                else:
                    break

        quad_max_index = sample.shape[0]
        if quad_max is not None:            
            for i in range(sample.shape[0]):
                if sample[i, 0] > quad_max:
                    quad_max_index = i 
                    break

        quad_bins = quad_max_index - quad_min_index

    else:
        quad_bins = 0
        quad_min_index = 0
        quad_max_index = 0

    if use_wp:
        sample = np.loadtxt("../%s/%s/%s" % (mock_dir, wp_dir, wp_files[0]))
        
        wp_min_index = 0
        if wp_min is not None:          
            for i in range(sample.shape[0]):
                if sample[i, 0] < wp_min:
                    wp_min_index = i + 1

                else:
                    break

        wp_max_index = sample.shape[0]
        if wp_max is not None:          
            for i in range(sample.shape[0]):
                if sample[i, 0] > wp_max:
                    wp_max_index = i
                    break

        wp_bins = wp_max_index - wp_min_index

    else:
        wp_bins = 0
        wp_min_index = 0
        wp_max_index = 0

    n = mono_bins + quad_bins + wp_bins
    mock_vectors = np.empty((M, n, 2))

    for i in range(M):
        wp_vector = np.loadtxt("../%s/%s/%s" % (mock_dir, wp_dir, wp_files[i]))
        mps_vector = np.loadtxt("../%s/%s/%s" % (mock_dir, mps_dir, mps_files[i]))

        mock_vectors[i, :, :] = np.concatenate((mps_vector[mono_min_index : mono_max_index,(0,1)],
                                                mps_vector[quad_min_index : quad_max_index,(0,2)],
            wp_vector[wp_min_index : wp_max_index, :]))

    mock_means = np.empty((n, 2))
    mock_means[: mono_bins, :] = np.loadtxt("../%s/%s" % (mock_dir, mean_mps_file))[mono_min_index : mono_max_index,
                                                                                    (0,1)]
    mock_means[mono_bins : mono_bins + quad_bins, :] = (np.loadtxt("../%s/%s" % (mock_dir, mean_mps_file))
                                                        [quad_min_index : quad_max_index,(0,3)])
    mock_means[-wp_bins:, :] = np.loadtxt("../%s/%s" % (mock_dir, mean_wp_file))[wp_min_index : wp_max_index, :2]

    cov_matrix = np.zeros((n, n))
    variances = np.zeros(n)

    for i in range(n):
        for j in range(n):
            for k in range(M):
                cov_matrix[i, j] += ((mock_vectors[k, i, 1] - mock_means[i, 1]) *
                                     (mock_vectors[k, j, 1] - mock_means[j, 1]))

                if i==j:
                    variances[i] += ((mock_vectors[k, i, 1] - mock_means[i, 1]) *
                                     (mock_vectors[k, i, 1] - mock_means[i, 1]))

            cov_matrix[i, j] = cov_matrix[i, j] / M

    variances = variances / M

    if plot_cov_matrix:
        plt.figure(figsize=(12,12))
        plt.matshow(cov_matrix)
        plt.colorbar()
        plt.savefig(output)

    if smooth_cov:
        cov_matrix = smooth_covariance_matrix(cov_matrix, divisions=[0, mono_bins, mono_bins + quad_bins,
                                                                     mono_bins + quad_bins + wp_bins])

    return cov_matrix

In [6]:
def calc_N_bins(binning):
    N_bins = []
    for dim in binning:
        N_bin = (dim[1] - dim[0]) / dim[2]
        N_bins.append(N_bin)
        
    return N_bins


def calc_step_size(binning):
    step_sizes = []
    for dim in binning:
        if dim[3] == 'log':
            step_size = (np.log10(dim[1]) - np.log10(dim[0])) / dim[2]
            
        else:
            step_size = (dim[1] - dim[0]) / dim[2]
        
        step_sizes.append(step_size)
        
    return step_sizes


def load_sparse(filename):
    return np.loadtxt(filename, dtype=np.float, delimiter='\t')


def fill_sparse(filename, binning):
    '''
    binning - array describing binning scheme
              [[s_min, s_max, N_bin, scaling],
              [mu_min, mu_max, N_bin, scaling],
              [reg_min, reg_max, N_bin, scaling],
              [reg_min, reg_max, N_bin, scaling]]
              where scaling is either 'log' or 'lin'
    '''
    binning = np.array(binning, dtype=[('min', 'f4'), ('max', 'f4'), ('N_bin', 'i4'), ('scaling', 'U5')])
    N_bins_tot = np.prod(binning['N_bin'])
    print(binning['N_bin'])
    print(N_bins_tot)
    step_sizes = calc_step_size(binning)
    
    print("Loading from file...")
    sparse_pair_counts = load_sparse(filename)
    print("Load complete.")
    
    
    pair_counts = np.zeros((N_bins_tot, len(binning)+1))
    # For now hard codes to 4 dimensional binning
    
    print("Writing binning information...")
    line_number = 0
    for i in range(binning[0][2]):
        for j in range(binning[1][2]):
            for k in range(binning[2][2]):
                for m in range(binning[3][2]):
                    indices = (i, j, k, m)
                    for n in range(4):
                        if binning[n][3] == 'log':
                            pair_counts[i*binning[0][2]+j*binning[1][2]+k*binning[2][2]+m, n] = (
                                np.power(10., np.log10(binning[n][0]) + float(indices[n]*step_sizes[n])))
                        else:
                            pair_counts[i*binning[0][2]+j*binning[1][2]+k*binning[2][2]+m, n] = (binning[n][0] +
                                                                                 float(indices[n]*step_sizes[n]))
                    line_number += 1
        print("%i lines written" % line_number)
    print("Binning information complete.")

    print("Adding non-zero bins...")
    for line in sparse_pair_counts:
        pair_counts[line[0]*N_bins[0]+line[1]*N_bins[1]+line[2]*N_bins[2]+line[3], 4] = line[4]
    print("Pair counts complete.")
        
    return pair_counts


def rebin(original, compression_factor, N_bins):
    rebinned = np.zeros((int(np.prod(N_bins)/compression_factor), 5))
    N_bins_new = [int(N_bins[0] / compression_factor), N_bins[1], N_bins[2], N_bins[3]]
    for i in range(N_bins[0]):
        i_new = int(i / compression_factor)
        for j in range(N_bins[1]):
            for k in range(N_bins[2]):
                for m in range(N_bins[3]):
                    if (i%compression_factor==0):
                        for n in range(4):
                            rebinned[i_new*N_bins_new[0]+j*N_bins[1]+k*N_bins[2]+m, n] = (
                                original[i*N_bins[0]+j*N_bins[1]+k*N_bins[2]+m, n])
                    
                    rebinned[i_new*N_bins_new[0]+j*N_bins[1]+k*N_bins[2]+m, 5] += (
                                original[i*N_bins[0]+j*N_bins[1]+k*N_bins[2]+m, 5])

In [53]:
binning = [(0.1, 60., 180, 'log'),
          (0., 1., 100, 'lin'),
          (0, 200, 200, 'lin'),
          (0, 200, 200, 'lin')]
binning = np.array(binning, dtype=[('min', 'f4'), ('max', 'f4'), ('N_bin', 'i4'), ('scaling', 'U5')])
N_bins_tot = np.prod(binning['N_bin'])
step_sizes = calc_step_size(binning)

pair_counts = np.zeros((N_bins_tot, len(binning)+1))

line_number = 0
start_time = dt.datetime.now()
print("Starting fill at", start_time)
for i in range(binning[0][2]):
    for j in range(binning[1][2]):
        for k in range(binning[2][2]):
            for m in range(binning[3][2]):
                indices = (i, j, k, m)
                for n in range(4):
                    if binning[n][3] == 'log':
                        pair_counts[i*binning[0][2]+j*binning[1][2]+k*binning[2][2]+m, n] = (
                            np.power(10., np.log10(binning[n][0]) + float(indices[n]*step_sizes[n])))
                    else:
                        pair_counts[i*binning[0][2]+j*binning[1][2]+k*binning[2][2]+m, n] = (binning[n][0] +
                                                                             float(indices[n]*step_sizes[n]))
                line_number += 1
    print("%i lines written" % line_number, "elapsed time", start_time-dt.datetime.now())

Starting fill at 2020-04-14 15:46:11.854656
4000000 lines written elapsed time -1 day, 23:55:38.917471
8000000 lines written elapsed time -1 day, 23:51:26.849672
12000000 lines written elapsed time -1 day, 23:47:19.430386
16000000 lines written elapsed time -1 day, 23:43:12.213933
20000000 lines written elapsed time -1 day, 23:39:04.295259
24000000 lines written elapsed time -1 day, 23:34:56.772399
28000000 lines written elapsed time -1 day, 23:30:34.887173


KeyboardInterrupt: 

In [7]:
binning = [(0.1, 60., 180, 'log'),
          (0., 1., 100, 'lin'),
          (0, 200, 200, 'lin'),
          (0, 200, 200, 'lin')]
binning = np.array(binning, dtype=[('min', 'f4'), ('max', 'f4'), ('N_bin', 'i4'), ('scaling', 'U5')])
N_bins_tot = np.prod(binning['N_bin'])
step_sizes = calc_step_size(binning)

col_1_basis = np.logspace(0.1, 60., 180, endpoint=False)
col_2_basis = np.linspace(0., 1., 100, endpoint=False)
col_3_basis = np.linspace(0, 200, 200, endpoint=False, dtype="i4")
col_4_basis = np.linspace(0, 200, 200, endpoint=False, dtype="i4")
col_5_basis = 0.

print("Starting Column 1", dt.datetime.now())
col_1 = np.tile(col_1_basis, (np.prod(binning['N_bin'][1:]),1))
col_1.flatten(order='F')
print("Finished Column 1", dt.datetime.now())

print("Starting Column 2", dt.datetime.now())
col_2_repeat = np.tile(col_2_basis, (np.prod(binning['N_bin'][2:]),1))
col_2_repeat.flatten(order='F')
col_2 = np.tile(col_2_repeat, binning['N_bin'][0])
print("Finished Column 2", dt.datetime.now())

print("Starting Column 3", dt.datetime.now())
col_3_repeat = np.tile(col_3_basis, (np.prod(binning['N_bin'][3:]),1))
col_3_repeat.flatten(order='F')
col_3 = np.tile(col_3_repeat, np.prod(binning['N_bin'][:2]))
print("Finished Column 3", dt.datetime.now())

print("Starting Column 4", dt.datetime.now())
col_4 = np.tile(col_4_basis, np.prod(binning['N_bin'][:3]))
print("Finished Column 4", dt.datetime.now())

print("Starting Column 5", dt.datetime.now())
col_5 = np.zeros(np.prod(binning['N_bin']))
print("Finished Column 5", dt.datetime.now())

print(col_1.shape(), col_2.shape(), col_3.shape(), col_4.shape(), col_5.shape())

Starting Column 1 2020-04-14 17:17:27.495981


KeyboardInterrupt: 

In [64]:
seq = [1,2,3,4]

arr = np.tile(seq, (2,1))
print(arr)

flat_arr = arr.flatten(order='F')
print(flat_arr)

[[1 2 3 4]
 [1 2 3 4]]
[1 1 2 2 3 3 4 4]


In [12]:
binning = [(0.1, 60., 180, 'log'),
          (0., 1., 100, 'lin'),
          (0, 200, 200, 'lin'),
          (0, 200, 200, 'lin')]

col_1_basis = np.logspace(np.log10(binning[0][0]), np.log10(binning[0][1]), binning[0][2], endpoint=False)
col_2_basis = np.linspace(binning[1][0], binning[1][1], binning[1][2], endpoint=False)
col_3_basis = np.linspace(binning[2][0], binning[2][1], binning[2][2], endpoint=False, dtype="i4")
col_4_basis = np.linspace(binning[3][0], binning[3][1], binning[3][2], endpoint=False, dtype="i4")
col_5_basis = 0. 

print(col_1_basis)
print(col_2_basis)
print(col_3_basis)
print(col_4_basis)
print(col_5_basis)

[ 0.1         0.10361775  0.10736639  0.11125064  0.11527542  0.1194458
  0.12376705  0.12824464  0.13288421  0.13769164  0.14267298  0.14783454
  0.15318283  0.15872461  0.16446687  0.17041688  0.17658214  0.18297045
  0.18958987  0.19644876  0.2035558   0.21091994  0.21855051  0.22645713
  0.23464979  0.24313884  0.251935    0.26104939  0.27049352  0.28027931
  0.29041912  0.30092577  0.31181252  0.32309313  0.33478185  0.34689343
  0.35944318  0.37244695  0.38592117  0.39988284  0.41434962  0.42933977
  0.44487222  0.46096661  0.47764324  0.4949232   0.5128283   0.53138117
  0.55060523  0.57052477  0.59116496  0.61255185  0.63471247  0.6576748
  0.68146786  0.70612168  0.73166743  0.75813736  0.7855649   0.8139847
  0.84343267  0.87394598  0.9055632   0.93832424  0.97227051  1.00744486
  1.04389173  1.08165717  1.12078886  1.16133624  1.20335053  1.24688479
  1.29199401  1.33873517  1.38716732  1.43735161  1.48935146  1.54323253
  1.59906288  1.65691304  1.71685607  1.7789677   1.84

In [13]:
empty_bins = np.load("../output/empty_jk_bins.npy")
print(empty_bins.shape)
print(empty_bins[:400])

(7200000, 5)
[[1.00e-01 0.00e+00 0.00e+00 0.00e+00 0.00e+00]
 [1.00e-01 0.00e+00 0.00e+00 1.00e+00 0.00e+00]
 [1.00e-01 0.00e+00 0.00e+00 2.00e+00 0.00e+00]
 [1.00e-01 0.00e+00 0.00e+00 3.00e+00 0.00e+00]
 [1.00e-01 0.00e+00 0.00e+00 4.00e+00 0.00e+00]
 [1.00e-01 0.00e+00 0.00e+00 5.00e+00 0.00e+00]
 [1.00e-01 0.00e+00 0.00e+00 6.00e+00 0.00e+00]
 [1.00e-01 0.00e+00 0.00e+00 7.00e+00 0.00e+00]
 [1.00e-01 0.00e+00 0.00e+00 8.00e+00 0.00e+00]
 [1.00e-01 0.00e+00 0.00e+00 9.00e+00 0.00e+00]
 [1.00e-01 0.00e+00 0.00e+00 1.00e+01 0.00e+00]
 [1.00e-01 0.00e+00 0.00e+00 1.10e+01 0.00e+00]
 [1.00e-01 0.00e+00 0.00e+00 1.20e+01 0.00e+00]
 [1.00e-01 0.00e+00 0.00e+00 1.30e+01 0.00e+00]
 [1.00e-01 0.00e+00 0.00e+00 1.40e+01 0.00e+00]
 [1.00e-01 0.00e+00 0.00e+00 1.50e+01 0.00e+00]
 [1.00e-01 0.00e+00 0.00e+00 1.60e+01 0.00e+00]
 [1.00e-01 0.00e+00 0.00e+00 1.70e+01 0.00e+00]
 [1.00e-01 0.00e+00 0.00e+00 1.80e+01 0.00e+00]
 [1.00e-01 0.00e+00 0.00e+00 1.90e+01 0.00e+00]
 [1.00e-01 0.00e+00 0.00e+0

In [8]:
arr_1 = [1,1,1,1]
arr_2 = [2,2,2,2]
arr_3 = [3,3,3,3]
arr_4 = [4,4,4,4]
arr_5 = [5,5,5,5]

full_arr = np.column_stack((arr_1, arr_2, arr_3, arr_4, arr_5))
print(full_arr)

[[1 2 3 4 5]
 [1 2 3 4 5]
 [1 2 3 4 5]
 [1 2 3 4 5]]


In [ ]:
pair_counts = np.zeros((N_bins_tot, len(binning)+1))

# print(col_4)

repeated_4 = np.tile(col_4, 200)

# print(repeated_4)

In [65]:
binning = [(0.1, 60., 180, 'log'),
          (0., 1., 100, 'lin'),
          (0, 200, 200, 'lin'),
          (0, 200, 200, 'lin')]
binning = np.array(binning, dtype=[('min', 'f4'), ('max', 'f4'), ('N_bin', 'i4'), ('scaling', 'U5')])
print(np.prod(binning['N_bin'][1:]))

4000000


In [47]:
binning = [(0.1, 60., 180, 'log'),
          (0., 1., 100, 'lin'),
          (0, 200, 200, 'lin'),
          (0, 200, 200, 'lin')]
pair_counts = load_sparse_pair_counts("../data/jackknife_pair_counts/DD_eBOSS_LRG_clustering_NGC_v7_rs_llist_jk.dat",
                                     binning)
print(pair_counts[:400])

[180 100 200 200]
720000000
Loading from file...
Load complete.
Writing binning information...
4000000 lines written
8000000 lines written


KeyboardInterrupt: 

In [ ]:
print(pair_counts.shape())
print(pair_counts[-200:])
print(pair_counts)

In [ ]:
data_ngc = Table.read("../data/eBOSS_LRG_clustering_NGC_v7.dat.fits")
rand_ngc = Table.read("../data/eBOSS_LRG_clustering_NGC_v7.ran.fits")

D_eff_ngc = np.sum(data_ngc['WEIGHT_CP']*data_ngc['WEIGHT_SYSTOT']*data_ngc['WEIGHT_NOZ']*data_ngc['WEIGHT_FKP'])
R_eff_ngc = np.sum(rand_ngc['WEIGHT_CP']*rand_ngc['WEIGHT_SYSTOT']*rand_ngc['WEIGHT_NOZ']*rand_ngc['WEIGHT_FKP'])

data_sgc = Table.read("../data/eBOSS_LRG_clustering_SGC_v7.dat.fits")
rand_sgc = Table.read("../data/eBOSS_LRG_clustering_SGC_v7.ran.fits")

D_eff_sgc = np.sum(data_sgc['WEIGHT_CP']*data_sgc['WEIGHT_SYSTOT']*data_sgc['WEIGHT_NOZ']*data_sgc['WEIGHT_FKP'])
R_eff_sgc = np.sum(rand_sgc['WEIGHT_CP']*rand_sgc['WEIGHT_SYSTOT']*rand_sgc['WEIGHT_NOZ']*rand_sgc['WEIGHT_FKP'])


# Research
textsize = 18
figsize = [12., 9.]
linewidth = 2
s_pow = 0.

plt.rc('xtick',labelsize=textsize)
plt.rc('ytick',labelsize=textsize)

DD_file_ngc = "../output/DD_eBOSS_LRG_clustering_NGC_v7_proj_llist.dat"
DR_file_ngc = "../output/DR_eBOSS_LRG_clustering_NGC_v7_proj_llist.dat"
RR_file_ngc = "../output/RR_eBOSS_LRG_clustering_NGC_v7_proj_llist.dat"

DD_file_sgc = "../output/DD_eBOSS_LRG_clustering_SGC_v7_proj_llist.dat"
DR_file_sgc = "../output/DR_eBOSS_LRG_clustering_SGC_v7_proj_llist.dat"
RR_file_sgc = "../output/RR_eBOSS_LRG_clustering_SGC_v7_proj_llist.dat"

In [ ]:
data_ngc = Table.read("../data/eBOSS_LRG_clustering_NGC_v7.dat.fits")
rand_ngc = Table.read("../data/eBOSS_LRG_clustering_NGC_v7.ran.fits")

D_eff_ngc = np.sum(data_ngc['WEIGHT_CP']*data_ngc['WEIGHT_SYSTOT']*data_ngc['WEIGHT_NOZ']*data_ngc['WEIGHT_FKP'])
R_eff_ngc = np.sum(rand_ngc['WEIGHT_CP']*rand_ngc['WEIGHT_SYSTOT']*rand_ngc['WEIGHT_NOZ']*rand_ngc['WEIGHT_FKP'])

data_sgc = Table.read("../data/eBOSS_LRG_clustering_SGC_v7.dat.fits")
rand_sgc = Table.read("../data/eBOSS_LRG_clustering_SGC_v7.ran.fits")

D_eff_sgc = np.sum(data_sgc['WEIGHT_CP']*data_sgc['WEIGHT_SYSTOT']*data_sgc['WEIGHT_NOZ']*data_sgc['WEIGHT_FKP'])
R_eff_sgc = np.sum(rand_sgc['WEIGHT_CP']*rand_sgc['WEIGHT_SYSTOT']*rand_sgc['WEIGHT_NOZ']*rand_sgc['WEIGHT_FKP'])


# Research
textsize = 18
figsize = [12., 9.]
linewidth = 2
s_pow = 0.

plt.rc('xtick',labelsize=textsize)
plt.rc('ytick',labelsize=textsize)

DD_file_ngc = "../output/DD_eBOSS_LRG_clustering_NGC_v7_rs_llist_v2.dat"
DR_file_ngc = "../output/DR_eBOSS_LRG_clustering_NGC_v7_rs_llist_v2.dat"
RR_file_ngc = "../output/RR_eBOSS_LRG_clustering_NGC_v7_rs_llist_v2.dat"

DD_file_sgc = "../output/DD_eBOSS_LRG_clustering_SGC_v7_rs_llist_v2.dat"
DR_file_sgc = "../output/DR_eBOSS_LRG_clustering_SGC_v7_rs_llist_v2.dat"
RR_file_sgc = "../output/RR_eBOSS_LRG_clustering_SGC_v7_rs_llist_v2.dat"